# Check the stock that out perform in the hypothetical portfolio


Analysing Best Performer stock in protfolio in last 3 month 
portfolio includes three sector i.e Banking, Pharma, and IT (BPI)
Sectorial allocation were discussed based on the the proportion of each sector. For example if market cap of sector B,P, I are 30,15,10 respectively then 54% (30/55) in B, 27% in P and remaining 9% in I
Then Invetesting Equal weight portfolio in each sector by inveting in top 4 company by market cap




(Source https://www.icicidirect.com/equity/index/nse/nifty-pharma/26769)

In [53]:
from fyers_apiv3 import fyersModel
import pyotp
import pandas as pd
import requests as req
import datetime as dt
from urllib.parse import parse_qs, urlparse
import time
#import seaborn as sea
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
credential = {
    'secret_key': 'xxxxxx',
    'client_id': 'xxxxxxx',
    'redirect_uri': 'https://trade.fyers.in/api-login/redirect-uri/index.html',
    'FY_ID': 'xxxxxx',
    'PIN': 'xxx',
    'TOTP': 'xxx'
}


In [54]:


def login(secret_key, client_id, redirect_uri, fy_id, pin, totp):
    BASE_URL = "https://api-t2.fyers.in/vagator/v2"
    BASE_URL_2 = "https://api-t1.fyers.in/api/v3"
    URL_SEND_LOGIN_OTP = BASE_URL + "/send_login_otp"
    URL_VERIFY_TOTP = BASE_URL + "/verify_otp"
    URL_VERIFY_PIN = BASE_URL + "/verify_pin"
    URL_TOKEN = BASE_URL_2 + "/token"
    URL_VALIDATE_AUTH_CODE = BASE_URL_2 + "/validate-authcode"
    res = req.post(url=URL_SEND_LOGIN_OTP, json={"fy_id": fy_id, "app_id": "2"}).json()
    if dt.datetime.now().second % 30 > 27: time.sleep(5)
    vari_topt = req.post(url=URL_VERIFY_TOTP,
                         json={"request_key": res['request_key'], "otp": pyotp.TOTP(totp).now()}).json()

    ses = req.Session()
    time.sleep(.5)
    vari_pin_payload = {"request_key": vari_topt["request_key"], "identity_type": "pin", "identifier": "xxxxxxxxxx"}

    vari_pin = req.post(url=URL_VERIFY_PIN, json=vari_pin_payload).json()

    ses.headers.update({"authorization": f"Bearer {vari_pin['data']['access_token']}"})
    token_payload3 = {
        "fyers_id": fy_id,
        "app_id": client_id[:-4],
        "redirect_uri": redirect_uri,
        "appType": "100", "code_challenge": "",
        "state": "None", "scope": "", "nonce": "",
        "response_type": "code", "create_cookie": True
    }

    token_ = ses.post(url=URL_TOKEN, json=token_payload3).json()

    url = token_['Url']
    parsed = urlparse(url)
    auth_code = parse_qs(parsed.query)['auth_code'][0]

    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type = 'code',
        grant_type = 'authorization_code'
    )
    session.set_token(auth_code)
    response = session.generate_token()
    access_token = response['access_token']
    fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")
    return access_token, fyers

access_token, fyers = login(
    credential.get("secret_key"),credential.get("client_id"),
    credential.get("redirect_uri"), credential.get("FY_ID"),
    credential.get("PIN"), credential.get("TOTP")
              )



1. Banking: Market Cap:
    Nifty Bank constitutes 12 stocks with total market cap 4,059,695.19,Cr.
2. Pharma:  Market Cap:
    Nifty Pharma constitutes 20 stocks with total market cap 1,750,884.26,Cr.
3. IT:      Market Cap = 
    Nifty IT constitutes 10 stocks with total market cap 3,790,322.85,Cr.


In [65]:
# fyers.get_profile()
# Sectorial_allocation
nifty_bank = 4059695.19 
pharma =  1750884.26
i_t =   3790322.85 
total =sum([nifty_bank, pharma, i_t])
nifty_bank_weight = round(nifty_bank/total,2)
pharma_weight = round(pharma/total,2)
i_t_weight = 1 - sum([nifty_bank_weight,pharma_weight])
initial_capital = 1000000


Get the Daily Historical Data of Top 4 firm by market cap in each sector


In [56]:
from time import sleep

def Historical(symbol:str):
    sleep(1)
    """
    This function is used to extract historical data from Fyers Server
    parameter:
        symbol: ticker name eg:TCS datatype:string
    """
    today  = dt.date.today() #yyyy-mm-dd
    start_day = today - dt.timedelta(90)
    print(today, start_day)
    data = {
        "symbol":f"NSE:{symbol.upper()}-EQ",
        "resolution":"D",
        "date_format":"1",
        "range_from":str(start_day),
        "range_to":str(today),
        "cont_flag":"1"
    }
    try:
        response = fyers.history(data=data)
        #print(response['candles'])
        #Lets strucutre the data
        data = pd.DataFrame(response['candles'], columns=["Date", "Open","High","Low","Close", "Volume"])
        data['Date'] = pd.to_datetime(data["Date"], unit='s')
        print(data[['Date',"Close"]].rename(columns={'Close':symbol.upper()}))
        return data[['Date',"Close"]].rename(columns={'Close':symbol.upper()})
    except:
        print(response, symbol)


nifty_bank_capital = round(nifty_bank_weight*initial_capital,2)
nifty_bank_portfolio_stock = ['HDFCBank','ICICIBank', 'AXISBANK', 'SBIN'] #top 4 company in banking sector 

pharma_capital = round(pharma_weight*initial_capital,2)
pharma_portfolio_stock =  ['SunPharma','DIVISLAB','Cipla','TORNTPHARM' ] #top 4 firm in pharma sector

i_t_capital = round(i_t_weight*initial_capital,2)
i_t_portfolio_stock = ['TCS','INFY','HCLTech','Wipro']#top 4 firm in IT sector
print(f"Capital Allocated in each sector are as follow:\n1. Banking: {nifty_bank_capital}\n2. Pharma: {pharma_capital}\n3. IT: {i_t_capital}")

#Portfolio Data
bank_portfoli_data = [Historical(a) for a in nifty_bank_portfolio_stock]
pharma_portfoli_data = [Historical(a) for a in pharma_portfolio_stock]
it_portfoli_data = [Historical(a) for a in i_t_portfolio_stock]

print(len(bank_portfoli_data), len(pharma_portfoli_data), len(it_portfoli_data))


Capital Allocated in each sector are as follow:
1. Banking: 420000.0
2. Pharma: 180000.0
3. IT: 400000.0
2024-09-07 2024-06-09
         Date  HDFCBANK
0  2024-06-10   1561.30
1  2024-06-11   1564.80
2  2024-06-12   1574.15
3  2024-06-13   1580.75
4  2024-06-14   1596.90
..        ...       ...
57 2024-09-02   1626.95
58 2024-09-03   1637.35
59 2024-09-04   1641.80
60 2024-09-05   1645.45
61 2024-09-06   1636.95

[62 rows x 2 columns]
2024-09-07 2024-06-09
         Date  ICICIBANK
0  2024-06-10    1123.80
1  2024-06-11    1118.20
2  2024-06-12    1119.55
3  2024-06-13    1107.35
4  2024-06-14    1105.65
..        ...        ...
57 2024-09-02    1229.95
58 2024-09-03    1247.70
59 2024-09-04    1236.35
60 2024-09-05    1235.95
61 2024-09-06    1208.15

[62 rows x 2 columns]
2024-09-07 2024-06-09
         Date  AXISBANK
0  2024-06-10   1200.00
1  2024-06-11   1194.60
2  2024-06-12   1187.90
3  2024-06-13   1174.65
4  2024-06-14   1181.05
..        ...       ...
57 2024-09-02   1188.80
58 

In [57]:
security_capital_allocation = {a.upper():100/4/100*nifty_bank_capital for a in nifty_bank_portfolio_stock}

[security_capital_allocation.update({a.upper(): 100/4/100*pharma_capital}) for a in pharma_portfolio_stock]
[security_capital_allocation.update({a.upper():100/4/100*i_t_capital}) for a in i_t_portfolio_stock]

security_capital_allocation

{'HDFCBANK': 105000.0,
 'ICICIBANK': 105000.0,
 'AXISBANK': 105000.0,
 'SBIN': 105000.0,
 'SUNPHARMA': 45000.0,
 'DIVISLAB': 45000.0,
 'CIPLA': 45000.0,
 'TORNTPHARM': 45000.0,
 'TCS': 100000.0,
 'INFY': 100000.0,
 'HCLTECH': 100000.0,
 'WIPRO': 100000.0}

In [58]:

portfoli_data = pd.concat(bank_portfoli_data+ pharma_portfoli_data+it_portfoli_data, axis=1)
portfoli_data = portfoli_data.transpose()
portfoli_data = portfoli_data.drop_duplicates().transpose()


In [59]:
portfoli_data.set_index(portfoli_data['Date'], inplace=True)
portfoli_data.drop(columns="Date", inplace=True)





In [60]:
cummulative_return_data = pd.DataFrame()

for a in portfoli_data.columns:
    cummulative_return_data[a] = portfoli_data[a].pct_change()
    cummulative_return_data[f"{a}"] = cummulative_return_data[a].cumsum()+1
    
    #cummulative_return_data.drop(columns=a, inplace=True)
    



In [61]:


px.line(cummulative_return_data, x=cummulative_return_data.index, y=cummulative_return_data.columns, title="Portfolio Securities Performance",
        labels={
                     "value": "Return (1.01 = 1%)",
                     
                     
                 },)


In [62]:
cum_return = cummulative_return_data.iloc[-1].to_dict()
ending_portfolio_value = 0
perforamce = {}
for  a in cummulative_return_data.iloc[-1].to_dict().keys():
    ending_portfolio_value+= cum_return.get(a)*security_capital_allocation.get(a)
    perforamce.update( {a:cum_return.get(a)*security_capital_allocation.get(a)})

In [63]:
ending_portfolio_value

1101179.279304006

In [64]:
perforamce_df = pd.DataFrame([list(perforamce.keys()), list(perforamce.values())])
perforamce_df = perforamce_df.transpose()


perforamce_df.rename(columns={0:"Securities", 1:"Individual securities holding value (in INR '000')"}, inplace=True)
perforamce_df.sort_values(by="Individual securities holding value (in INR '000')", ascending=False, inplace=True)
px.bar(perforamce_df, x='Securities', y="Individual securities holding value (in INR '000')", title="Best Performer",)